In [97]:
import os
import sys
import importlib
import numpy as np
import pandas as pd
import nibabel as nib

sys.path.append('/host/verges/tank/data/daniel/00_commonUtils/00_code/genUtils/')
import projectUtils as prjUtils
import stitchSurfs as stitch

In [98]:
# PATHS
glsr_lbl = {
    "parcellationName": "Glasser360",
    "parcName": "glsr",
    "type": "label",
    "pth_label_gii":"/host/verges/tank/data/daniel/parcellations/glasser/glasser-360_conte69_rh.label.gii",
    "pth_csv": "/host/verges/tank/data/daniel/parcellations/glasser/glasser_idx-lbls.csv",
    "csv_idx_label_colNames": ["idx", "LName"],
}

dk25_lbl = {
    "parcellationName": "DeKraker25",
    "parcName": "DK25",
    "type": "label",
    "pth_label_gii": "/host/verges/tank/data/daniel/parcellations/DeKraker25/sub-bigbrain_hemi-R_label-hipp_den-0p5mm_DeKraker25.label.gii",
    "pth_csv": "/host/verges/tank/data/daniel/parcellations/DeKraker25/DK25_details.csv",
    "csv_idx_label_colNames": ["idx", "label"],
}

tmplt_path_root = "/host/verges/tank/data/daniel/04_inVivoHistology/code/resources/"


In [99]:
# STITCH LABELS
importlib.reload(stitch)
gi = stitch.stitchLabels(ctx_lbl=glsr_lbl, hipp_lbl=dk25_lbl, 
                          outPth=tmplt_path_root)

[stitch.resolve_OverlapLblVals] Found 25 overlapping label values for cortical and hippocampal parcelletations.
	Offsetting hippocampal labels by 181.
	Saved merged label CSV to: /host/verges/tank/data/daniel/04_inVivoHistology/code/resources/stitch_lblValDetails_ctx-glsr_hipp-DK25_05Feb2026-1457.csv
	Saved stitched label Gifti to: /host/verges/tank/data/daniel/04_inVivoHistology/code/resources/stitch_lblVals_ctx-glsr_hipp-DK25_05Feb2026-1457.label.gii


In [108]:
# MAKE MASK BASED ON LABEL VALUES
importlib.reload(prjUtils)

lbl_gii_pth = os.path.join(tmplt_path_root, "stitch_lblVals_ctx-glsr_hipp-DK25_05Feb2026-1431.label.gii")
lbl_csv_pth = os.path.join(tmplt_path_root, "stitch_lblValDetails_ctx-glsr_hipp-DK25_05Feb2026-1431.csv")
mask = prjUtils.make_mask(lbl_tmplt_pth=lbl_gii_pth, csv_pth=lbl_csv_pth, 
                   label_col=['mesialTemporalMask'], label_vals=[[1]], 
                   savePath=tmplt_path_root, 
                   saveName="stitch_ctx-glsr_hipp-DK25_05Feb2026_mesialTemporal_mask")

mask_data = nib.load(mask)
print(type(mask_data))
n_true = np.sum(mask_data.darrays[0].data == 1)
print(f"Number of True values in mask: {n_true}")

# APPLY MASK TO LABEL FILE
importlib.reload(prjUtils)

mask = "stitch_ctx-glsr_hipp-DK25_05Feb2026_mesialTemporal_mask.gii"

prjUtils.apply_mask_to_stitchedGii(input_file = os.path.join(tmplt_path_root, lbl_gii_pth), 
                  mask_file = os.path.join(tmplt_path_root, mask), 
                  output_file = os.path.join(tmplt_path_root, "stitch_lblVals_ctx-glsr_hipp-DK25_05Feb2026_mesialTemporal_masked.label.gii"), 
                  apply_to_labels=True)


Mask saved to: /host/verges/tank/data/daniel/04_inVivoHistology/code/resources/stitch_ctx-glsr_hipp-DK25_05Feb2026_mesialTemporal_mask.gii
<class 'nibabel.gifti.gifti.GiftiImage'>
Number of True values in mask: 8477
Saved: /host/verges/tank/data/daniel/04_inVivoHistology/code/resources/stitch_lblVals_ctx-glsr_hipp-DK25_05Feb2026_mesialTemporal_masked.label.gii
Vertices: 37741 → 8477


'/host/verges/tank/data/daniel/04_inVivoHistology/code/resources/stitch_lblVals_ctx-glsr_hipp-DK25_05Feb2026_mesialTemporal_masked.label.gii'

In [ ]:
# [TEST] APPLY MASK TO SURFACE
pt_root = "/host/verges/tank/data/daniel/04_inVivoHistology/data/test/PNI/sub-Pilot014_ses-a1/surfs/"
commonName = "sub-Pilot014_ses-a1_hemi-R_ctxSurf-fsLR-32k_ctxLbl-white_hippSurf-den-0p5mm_hippLbl-outer_stitched"
surf = f"{commonName}.surf.gii"
prjUtils.apply_mask_to_stitchedGii(input_file = os.path.join(pt_root, surf), 
                  mask_file = os.path.join(tmplt_path_root, mask), 
                  output_file = os.path.join(pt_root, f"{commonName}_mesialTemporal_masked.surf.gii"))


Saved: /host/verges/tank/data/daniel/04_inVivoHistology/data/test/PNI/sub-Pilot014_ses-a1/surfs/sub-Pilot014_ses-a1_hemi-R_ctxSurf-fsLR-32k_ctxLbl-white_hippSurf-den-0p5mm_hippLbl-outer_stitched_mesialTemporal_masked.surf.gii
Vertices: 37741 → 8477


'/host/verges/tank/data/daniel/04_inVivoHistology/data/test/PNI/sub-Pilot014_ses-a1/surfs/sub-Pilot014_ses-a1_hemi-R_ctxSurf-fsLR-32k_ctxLbl-white_hippSurf-den-0p5mm_hippLbl-outer_stitched_mesialTemporal_masked.surf.gii'